# Setting Up

In [1]:
# !pip install pyomo
# !pip install gurobipy
# !pip install xlsxwriter

import pandas as pd
import numpy as np
import itertools
import pyomo.environ as pe
import pyomo.gdp as gdp
import gurobipy
import pyomo.opt as po

# Data

## Sets

In [2]:
list(range(1, 11))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [3]:
sets = {
    # "S"     : ["x1", "x2", ...],  # Indices for supply geogpraphies
    # "X"     : ["x1", "x2", ...],  # Indices for plant site locations
    # "M"     : ["m1", "m2", ...],  # Indices for markets
    "T"       : [1],  # Indices for periods (10 years)
    "F"       : ["Bstraw", "Wstraw"],  # Indices for feedstocks
    "C"       : [],  # Indices for components
    "C_Solid" : ["Cellulose", "Hemicellulose", "Lignin"],  # Indices for solid components
    "C_Liquid": ["Xylose", "Glucose"],  # Indices for liquid components
    
    "BP"      : ['Succinic Acid'
                 #,'Glycolic Acid','Formic Acid','Acetic Acid','Phenolic','Furfural','HMF'
                ],  # Indices for by-products
    
    "U"       : ['H2SO4 kg'
                 #,'Water kg','Ammonia kg','Ionic Liquid mat. kg','Lime mat. kg'
                ],  # Indices for utilities
    
    "K"       : [],  # Indices for process options
    "K_prep"  : ['MILL3mm'
                 #,'MILL1.4mm','MILL10mm','MILL0.853mm','MILL0.15mm'
                ],  # Indices for preprocessing options
    
    "K_pret"  : ['Dilute Acid','LHW','AFEX','Steam Explosion','Lime','Ionic Liquid'],  # Indices for pretreatment options
    
    "K_conv"  : ['SHF','SSF','SHCF','SSCF'],  # Indices for fermentation options
    
    "K_pur"   : ["Reactive Extraction", 'Precipitation', 'Electrodialysis', 'Direct Crystalization', 'Azeotropic Dist', 'Extraction', 'Absorption'],  # Indices for purification options
    
    "J"       : ["Strain1","Strain2","Strain3","Strain4","Strain5","Strain6 ","Strain7","Strain8","Strain9" 
                 #"8G-3", "SPT2OE", "Y2034", "Angle", "CRD51", "MD4", "TP50", "SR8/TFP3-XI76", "SR8/TFP3-XI76"
                ],  # Indices for fermentation strains
    
    "P"       : ['Ethanol', 'Lactic Acid', 'Succinic Acid'],  # Indices for final products
    
    "Q"       : [10**6]  # Indices for technology capacity levels
}

sets["C"] = sets["C_Solid"]  + sets["C_Liquid"] 
sets["K"] = sets["K_prep"]  + sets["K_pret"] + sets["K_conv"] + sets["K_pur"]

## Parameters

Most parameters are indexed across different sets. To improve decision-maker interaction, the parametric tables over different sets are created with all possible combinations of the sets but empty values for the parameter and exported as excel files for easy inputs

In [4]:
# Funciton for create the dataframe from the cartesian product of specific sets
def create_parametric_df(sets):
    df = pd.DataFrame(itertools.product(*sets.values()), columns=sets.keys())
    # drop duplicated rows
    df = df.drop_duplicates().reset_index(drop=True)
    # Create an empty value columns
    df.loc[:, 'Value'] = np.nan
    # Input 0 in impossible combinations (relevant for yields as you can't convert cellulose into cellulose)
    # Identify rows where a value in a column is equal to a value in another column for all columns
    equal_values_rows = df[df.apply(lambda row: len(set(row)) < len(row), axis = 1)]
    # Cahnge value for identified rows
    df.loc[equal_values_rows.index, "Value"] = 0
    return df

def create_compatability_dic():
    
    # Dictionary to store the DataFrames
    dfs = {}

    # Compatability Preprocessing with Pretreatment
    dfs['Compatibility_Pretreatment'] = create_parametric_df({'K_prep' : sets['K_prep'], 'K_pret' : sets['K_pret']})
    
    # Compatability Pretreatment, Conversion, Fermentation Strain, Component and Product   
    dfs['Conversion_Matrix_Pret_Conv'] = create_parametric_df({'K_pret': sets['K_pret'], 'K_conv': sets['K_conv']})
    dfs['Conversion_Matrix_C_Conv'] = create_parametric_df({'C': sets['C'], 'K_conv': sets['K_conv']})
    dfs['Conversion_Matrix_Conv_J'] = create_parametric_df({'K_conv': sets['K_conv'], 'J': sets['J']})
    dfs['Conversion_Matrix_J_P'] = create_parametric_df({'J': sets['J'], 'P': sets['P']})
    dfs['Conversion_Matrix_C_P'] = create_parametric_df({'C': sets['C'], 'P': sets['P']})
    dfs['Conversion_Matrix_Conv_P'] = create_parametric_df({'K_conv': sets['K_conv'], 'P': sets['P']})
    
    dfs['Compatibility_Purification'] = create_parametric_df({'P': sets['P'], 'K_pur': sets['K_pur']})

    return dfs

def create_parametric_dic(compat_dfs):
    # Dictionary to store the DataFrames
    dfs = {}

    # Generate DataFrames and store them in the dictionary
    dfs['Cost_of_Technology'] = create_parametric_df({'K': sets['K'], 'Q': sets['Q']})
    dfs['MinMax_Flows'] = create_parametric_df({'Flow': ['IN', 'OUT'], 'MinMax': ['Min', 'Max'], 'K': sets['K'], 'Q': sets['Q']})
    dfs['Utility_Consumption'] = create_parametric_df({'U': sets['U'], 'K': sets['K']})
    dfs['Byproduct_Production'] = create_parametric_df({'BP': sets['BP'], 'K': sets['K']})
    dfs['Feedstock_Composition'] = create_parametric_df({'C': sets['C'], 'F': sets['F']})
    
    dfs['Compatibility_Pretreatment'] = compat_dfs['Compatibility_Pretreatment']
    dfs['Yield_Pretreatment'] = create_parametric_df({'C': sets['C'], 'K_pret': sets['K_pret']})
    
    # Compatability Conversion
    df = pd.merge(compat_dfs['Conversion_Matrix_Pret_Conv'], compat_dfs['Conversion_Matrix_C_Conv'], on = "K_conv", how = "outer")
    df["Value"] = df.Value_x * df.Value_y
    df = df[df["Value"] != 0].drop(['Value_x', 'Value_y'], axis = 1)
    df = df.merge(compat_dfs['Conversion_Matrix_Conv_J'], on = "K_conv", how = "left")
    df["Value"] = df.Value_x * df.Value_y
    df = df[df["Value"] != 0].drop(['Value_x', 'Value_y'], axis = 1)
    df = df.merge(compat_dfs['Conversion_Matrix_J_P'], on = "J", how = "left")
    df["Value"] = df.Value_x * df.Value_y
    df = df[df["Value"] != 0].drop(['Value_x', 'Value_y'], axis = 1)
    df = df.merge(compat_dfs['Conversion_Matrix_C_P'], on = ["C", "P"], how = "left")
    df["Value"] = df.Value_x * df.Value_y
    df = df[df["Value"] != 0].drop(['Value_x', 'Value_y'], axis = 1)
    df = df.merge(compat_dfs['Conversion_Matrix_Conv_P'], on = ["K_conv", "P"], how = "left")
    df["Value"] = df.Value_x * df.Value_y
    df = df[df["Value"] != 0].drop(['Value_x', 'Value_y'], axis = 1)

    df = df[['C', 'K_pret', 'K_conv', 'J', 'P', 'Value']]

    dfs['Compatibility_Conversion'] = df
    
    dfs['Yield_Hydrolysis'] = df[['K_conv', 'K_pret', 'C', 'P', 'Value']].drop_duplicates().reset_index(drop=True).sort_values(['K_conv', 'K_pret', 'C', 'P'])
    dfs['Yield_Conversion'] = df[[ 'P', 'C', 'K_conv', 'J', 'Value']].drop_duplicates().reset_index(drop=True).sort_values(['J', 'P', 'C', 'K_conv'])
    
    dfs['Solid_Loading'] = create_parametric_df({'K_conv' : sets['K_conv']})

    dfs['Compatibility_Purification'] = compat_dfs['Compatibility_Purification']
    
    dfs['Yield_Purification'] = dfs['Compatibility_Purification'][dfs['Compatibility_Purification'].Value != 0].copy()
    dfs['Depreciation_Factor'] = create_parametric_df({'T' : sets['T']})
    dfs['Operating_Cost'] = dfs['Depreciation_Factor'].copy()
    dfs['Maintenance_Cost'] = dfs['Depreciation_Factor'].copy()
    dfs['Demand'] = create_parametric_df({'P': sets['P'], 'T': sets['T']})
    dfs['Supply'] = create_parametric_df({'F': sets['F'], 'T': sets['T']})
    dfs['Price_F'] = create_parametric_df({'F': sets['F'], 'T': sets['T']})
    dfs['Price_U'] = create_parametric_df({'U': sets['U'], 'T': sets['T']})
    dfs['Price_P'] = create_parametric_df({'P': sets['P'], 'T': sets['T']})

    return dfs

def create_compatability_excel(xls_path = "CompatabilityMatrices.xlsx"):
    dfs = create_compatability_dic()
    writer = pd.ExcelWriter(xls_path, engine='xlsxwriter')
    for name, df in dfs.items():
        df.to_excel(writer, sheet_name=name, index = False)
    writer.close()
    writer.handles = None
    return dfs

def read_compatability_excel(xls_path = "CompatabilityMatricesInputs.xlsx"):

    dfs = create_compatability_dic()
    excel_dfs = {}
    
    # Read excel
    xls = pd.ExcelFile(xls_path)
    for name, df in dfs.items():
        excel_df = pd.read_excel(xls, sheet_name=name)
        try:
            excel_df = excel_df.drop("Unnamed: 0", axis = 1)
        except:
            pass
        
        # concat excel df with full df
        df = pd.concat([excel_df, dfs[name]]).reset_index(drop=True)
        # keep first appearence (coming from excel df) of duplicated values in all columns except Value
        df = df.drop_duplicates(subset = df.columns.difference(['Value']), keep='first').reset_index(drop=True)
        # change all NA to 0 excpet for solid loading
        df.Value = 1 if name in ["Solid_Loading"] else df.Value.fillna(0)
        
        excel_dfs[name] = df
    
    xls.close()
    xls.handles = None
    
    return excel_dfs

def create_parametric_excel(xls_path = "ParametricTables.xlsx", compat = None):
    
    if compat is None:
        # Try to get input tables
        print("No compatabiity tables given. Attempting to retrieve from CompatabilityMatricesInputs.xlsx")
        compat_dfs = read_compatability_excel()
    elif type(compat) is str:
        compat_dfs = read_compatability_excel(compat)
    else:
        compat_dfs = compat
    
    dfs = create_parametric_dic(compat_dfs)
    writer = pd.ExcelWriter(xls_path, engine='xlsxwriter')
    for name, df in dfs.items():
        df.to_excel(writer, sheet_name=name, index = False)
    writer.close()
    writer.handles = None
    return dfs

def read_parametric_excel(xls_path, compat = None):
    
    if compat is None:
        # Try to get input tables
        print("No compatabiity tables given. Attempting to retrieve from CompatabilityMatricesInputs.xlsx")
        compat_dfs = read_compatability_excel()
    elif type(compat) is str:
        compat_dfs = read_compatability_excel(compat)
    else:
        compat_dfs = compat
        
    # Create empty dic to force parameters to be 0 when user accidently forgets certain inputs
    dfs = create_parametric_dic(compat_dfs)
    
    # Read excel
    excel_dfs = {}
    xls = pd.ExcelFile(xls_path)
    for name, df in dfs.items():
        excel_df = pd.read_excel(xls, sheet_name=name)
        try:
            excel_df = excel_df.drop("Unnamed: 0", axis = 1)
        except:
            pass
        # concat excel df with full df
        df = pd.concat([excel_df, dfs[name]]).reset_index(drop=True)
        # keep first appearence (coming from excel df) of duplicated values in all columns except Value
        df = df.drop_duplicates(subset = df.columns.difference(['Value']), keep='first').reset_index(drop=True)
        # change all NA to 0 excpet for solid loading
        df.Value = 1 if name in ["Solid_Loading"] else df.Value.fillna(0)
        
        excel_dfs[name] = df
    xls.close()
    xls.handles = None
    return excel_dfs

In [5]:
dfs = create_parametric_excel("ParametricTables-V1.xlsx")
print(dfs['Yield_Conversion'].shape)
dfs['Yield_Conversion'].head()

No compatabiity tables given. Attempting to retrieve from CompatabilityMatricesInputs.xlsx
(37, 5)


,P,C,K_conv,J,Value
0,Ethanol,Cellulose,SHF,Strain1,1.0
9,Ethanol,Hemicellulose,SHF,Strain1,1.0
18,Ethanol,Xylose,SHF,Strain1,1.0
1,Ethanol,Cellulose,SHF,Strain2,1.0
27,Ethanol,Cellulose,SSF,Strain2,1.0


In [502]:
dfs = read_parametric_excel("ParametricTablesInputs-V1.xlsx")
print(dfs['Yield_Conversion'].shape)
dfs['Yield_Conversion'][dfs['Yield_Conversion'].Value != 0].head()

No compatabiity tables given. Attempting to retrieve from CompatabilityMatricesInputs.xlsx
(37, 5)


,P,C,K_conv,J,Value
0,Ethanol,Cellulose,SHF,Strain1,0.90
5,Ethanol,Hemicellulose,SHF,Strain2,0.89
6,Ethanol,Xylose,SHF,Strain2,0.89
7,Ethanol,Xylose,SSF,Strain2,0.89
8,Ethanol,Cellulose,SHCF,Strain3,0.95


In [503]:
dfs['Yield_Hydrolysis'].head()

,K_conv,K_pret,C,P,Value
0,SHCF,AFEX,Cellulose,Ethanol,0.7000
1,SHCF,AFEX,Hemicellulose,Ethanol,0.7000
2,SHCF,Dilute Acid,Cellulose,Ethanol,1.0123
3,SHCF,Dilute Acid,Hemicellulose,Ethanol,0.7549
4,SHCF,Ionic Liquid,Cellulose,Ethanol,0.4560


# Model

## Initialiazation

### Model

In [518]:
model = pe.ConcreteModel()
bigM = 10**100

### Sets

In [519]:
# Initialize the sets based on the 'sets' dictionary
model.T = pe.Set(initialize=sets['T'])  # Indices for periods
model.F = pe.Set(initialize=sets['F'])  # Indices for feedstocks

model.C_Solid = pe.Set(initialize=sets['C_Solid'])  # Indices for solid components
model.C_Liquid = pe.Set(initialize=sets['C_Liquid'])  # Indices for liquid components
model.C = model.C_Solid | model.C_Liquid # Indices for all components

model.BP = pe.Set(initialize=sets['BP'])  # Indices for by-products
model.U = pe.Set(initialize=sets['U'])  # Indices for utilities

model.K_prep = pe.Set(initialize=sets['K_prep'])  # Indices for preprocessing options
model.K_pret = pe.Set(initialize=sets['K_pret'])  # Indices for pretreatment options
model.K_conv = pe.Set(initialize=sets['K_conv'])  # Indices for fermentation options
model.K_pur = pe.Set(initialize=sets['K_pur'])  # Indices for purification options
model.K = model.K_prep | model.K_pret | model.K_conv | model.K_pur # Indices for all process options

model.J = pe.Set(initialize=sets['J'])  # Indices for fermentation strains
model.P = pe.Set(initialize=sets['P'])  # Indices for final products
model.Q = pe.Set(initialize=sets['Q'])  # Indices for technology capacity levels

# Auxiliary Sets
model.Flow = pe.Set(initialize=["IN", "OUT"])
model.MinMax = pe.Set(initialize=["Min", "Max"])
model.FCP = pe.Set(initialize=list(sets['F']) + list(sets['C']) + list(sets['P']))

### Parameters

In [520]:
# Initialize parameters as defined in dfs dictionary: doenst work yet, cant fin a way to initialize coorectly last line

# A function to determine the indexing sets of a given dataframe based on the columns
def determine_indexing_sets(df):
    return tuple(col for col in df.columns if col != "Value")

def initialize_parameters(parameter_dfs):
    # Loop through the paramneter dfs and initialize parameters
    for df_name, df in parameter_dfs.items():
        index_sets = determine_indexing_sets(df)
        # Pyomo only accepts dict {(colum A, B, C) : Value, next row}
        param_init_dict = df.set_index([*index_sets]).Value.to_dict()

        model_sets = tuple(getattr(model, index_set) for index_set in index_sets)
        setattr(model, df_name, pe.Param(model.P, model.T, initialize=param_init_dict, within='Any'))

#initialize_parameters(read_parametric_excel("ParametricTables.xlsx"))

## Defining Variables

### Binary Variables

### Continuous Variables

In [521]:
# Parameters

# Feedstock_Composition
parameter_dict = dfs['Feedstock_Composition'].set_index(['C', 'F']).Value.to_dict()
model.Feedstock_Composition = pe.Param(model.C, model.F, initialize=parameter_dict, default=0)
# Yield_Pretreatment
parameter_dict = dfs['Yield_Pretreatment'].set_index(['C', 'K_pret']).Value.to_dict()
model.Yield_Pretreatment = pe.Param(model.C, model.K_pret, initialize=parameter_dict, default=0)
# Yield_Conversion
parameter_dict = dfs['Yield_Conversion'][['P', 'C', 'K_conv', 'J', "Value"]].set_index(['P', 'C', 'K_conv', 'J']).Value.to_dict()
model.Yield_Conversion = pe.Param(model.P, model.C, model.K_conv, model.J, initialize=parameter_dict, default=0)
# Yield_Purification 
parameter_dict = dfs['Yield_Purification'].set_index(['P', 'K_pur']).Value.to_dict()
model.Yield_Purification = pe.Param(model.P, model.K_pur, initialize=parameter_dict, default=0)
# Demand
parameter_dict = dfs['Demand'].set_index(['P', 'T']).Value.to_dict()
model.Demand = pe.Param(model.P, model.T, initialize=parameter_dict, default=0)
# Supply
parameter_dict = dfs['Supply'].set_index(['F', 'T']).Value.to_dict()
model.Supply = pe.Param(model.F, model.T, initialize=parameter_dict, default=0)

# Decision Variables (inflows)

# Quantity of feedstock f going into preprocessing k_prep at period t
model.F_in_prep = pe.Var(model.F, model.K_prep, model.T, domain=pe.NonNegativeReals)
# Quantity of feedstock f going out of preprocessing k_prep at period t
model.F_out_prep = pe.Var(model.F, model.K_prep, model.T, domain=pe.NonNegativeReals)

# Quantity of feedstock f going into pretreatment k_pret at period t
model.F_in_pret = pe.Var(model.F, model.K_pret, model.T, domain=pe.NonNegativeReals)
# Quantity of component c going out of pretreatment k_pret at period t
model.F_out_pret = pe.Var(model.C, model.K_pret, model.T, domain=pe.NonNegativeReals)

# Quantity of component c going into conversion k_conv with fermentation strain j at period t to produce product p
model.F_in_conv = pe.Var(model.C, model.P, model.K_conv, model.J, model.T, domain=pe.NonNegativeReals)
# Quantity of product p going out of conversion k_conv with fermentation strain j at period t from component c
model.F_out_conv = pe.Var(model.C, model.P, model.K_conv, model.J, model.T, domain=pe.NonNegativeReals)

# Quantity of product p going into purification k_conv at period t
model.F_in_pur = pe.Var(model.P, model.K_pur, model.T, domain=pe.NonNegativeReals)

# Quantity of product p going out of purification k_pur at period t
model.F_out_pur = pe.Var(model.P, model.K_pur, model.T, domain=pe.NonNegativeReals)






In [522]:
model.Yield_Conversion["Ethanol", "Cellulose", "SHF", "Strain1"]

0.9

In [523]:
model.Supply.pprint()

Supply : Size=2, Index=Supply_index, Domain=Any, Default=0, Mutable=False
    Key           : Value
    ('Bstraw', 1) :   5.0
    ('Wstraw', 1) :   5.0


## Constraint Formulation

### Operational Comstraints (2nd Stage): General

In [524]:
# Mass Balances

model.Mass_Balances = pe.ConstraintList()
for t in model.T:
    # Between Processes
    for f in model.F:
        model.Mass_Balances.add(model.F_in_prep[f, k_prep, t] <= model.Supply[f, t])
        model.Mass_Balances.add(sum(model.F_in_pret[f, k, t] for k in model.K_pret) == sum(model.F_out_prep[f, k, t] for k in model.K_prep))
    for c in model.C:
        model.Mass_Balances.add(sum(model.F_in_conv[c, p, k, j, t] for p in model.P for k in model.K_conv for j in model.J) == sum(model.F_out_pret[c, k, t] for k in model.K_pret))     
    
    for p in model.P:
        model.Mass_Balances.add(sum(model.F_in_pur[p, k, t] for k in model.K_pur) == sum(model.F_out_conv[c, p, k, j, t] for c in model.C for k in model.K_conv for j in model.J))
        model.Mass_Balances.add(sum(model.F_out_pur[p, k, t] for k in model.K_pur) <= model.Demand[p, t])

    # In Processes (Transformations)
    for f in model.F:
        for k_prep in model.K_prep:
            model.Mass_Balances.add(model.F_out_prep[f, k_prep, t]== model.F_in_prep[f, k_prep, t])
    for k_pret in model.K_pret:
        for c in model.C:
            model.Mass_Balances.add(model.F_out_pret[c, k_pret, t] == sum(model.F_in_pret[f, k_pret, t]  * model.Feedstock_Composition[c, f] for f in model.F) * model.Yield_Pretreatment[c, k_pret])
    for p in model.P:
        for c in model.C:
            for j in model.J:
                for k_conv in model.K_conv:
                    model.Mass_Balances.add(model.F_out_conv[c, p, k_conv, j, t] == model.F_in_conv[c, p, k_conv, j, t] * model.Yield_Conversion[p, c, k_conv, j])
        for k_pur in model.K_pur:
            model.Mass_Balances.add(model.F_out_pur[p, k_pur, t] == model.F_in_pur[p, k_pur, t] * model.Yield_Purification[p, k_pur])


In [525]:
model.Yield_Pretreatment.pprint()

Yield_Pretreatment : Size=30, Index=Yield_Pretreatment_index, Domain=Any, Default=0, Mutable=False
    Key                                  : Value
                   ('Cellulose', 'AFEX') :   0.9941
            ('Cellulose', 'Dilute Acid') :    0.895
           ('Cellulose', 'Ionic Liquid') :   0.8746
                    ('Cellulose', 'LHW') :   0.9221
                   ('Cellulose', 'Lime') :    0.968
        ('Cellulose', 'Steam Explosion') :   0.8068
                     ('Glucose', 'AFEX') :    0.019
              ('Glucose', 'Dilute Acid') :   0.1133
             ('Glucose', 'Ionic Liquid') :      0.0
                      ('Glucose', 'LHW') :      0.0
                     ('Glucose', 'Lime') :  0.00268
          ('Glucose', 'Steam Explosion') :   0.0233
               ('Hemicellulose', 'AFEX') :   0.6491
        ('Hemicellulose', 'Dilute Acid') :    0.026
       ('Hemicellulose', 'Ionic Liquid') :   0.4256
                ('Hemicellulose', 'LHW') :      0.0
               ('Hem

## Objective Functions

In [526]:
model.Objective_function = pe.Objective(rule = sum(model.Demand[p, t] - sum((model.F_out_pur[p, k, t] for k in model.K_pur)) for p in model.P for t in model.T),
                                        sense = pe.minimize)


In [527]:
model.Demand.pprint()

Demand : Size=3, Index=Demand_index, Domain=Any, Default=0, Mutable=False
    Key                  : Value
          ('Ethanol', 1) :   5.0
      ('Lactic Acid', 1) :   5.0
    ('Succinic Acid', 1) :   5.0


# Model Solve

In [529]:
solver = po.SolverFactory("gurobi", olver_io="python")
solver_parameters = "ResultFile=iismodel.ilp" # write an ILP file to print the IIS
results = solver.solve(model, options_string=solver_parameters)
results.write()
print_vars()


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x1
  Lower bound: 8.584651087792189
  Upper bound: 8.584651087792189
  Number of objectives: 1
  Number of constraints: 608
  Number of variables: 1156
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 1156
  Number of nonzeros: 1833
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Model w

In [297]:
import json
var_mapping = {}
index = 1  # Start index at 1, assuming Gurobi's first variable is x1

for v in model.component_objects(pe.Var, active=True):
    for index_within_v in v:
        var_mapping[f"x{index}"] = f"{v.name}[{index_within_v}]"
        index += 1

with open('var_mapping.json', 'w') as fp:
    json.dump(var_mapping, fp)
    
constraint_mapping = {}
index = 1  # Start index at 1, assuming Gurobi's first variable is x1

for v in model.component_objects(pe.Constraint, active=True):
    for index_within_v in v:
        constraint_mapping[f"x{index}"] = f"{v.name}[{index_within_v}]"
        index += 1
        
with open('constraint_mapping.json', 'w') as fp:
    json.dump(constraint_mapping, fp)

In [298]:
# Sort the dictionary by length of the keys in descending order
var_mapping = dict(sorted(var_mapping.items(), key=lambda item: len(item[0]), reverse=True))
constraint_mapping = dict(sorted(constraint_mapping.items(), key=lambda item: len(item[0]), reverse=True))

# Read the file content
with open("iismodel.ilp", "r") as file:
    iis_output = file.read()

# Replace Gurobi names with Pyomo names in constraints
for gurobi_name, pyomo_name in constraint_mapping.items():
    iis_output = iis_output.replace(f"{gurobi_name}_:", f"{pyomo_name}:")

# Replace Gurobi names with Pyomo names
for gurobi_name, pyomo_name in var_mapping.items():
    iis_output = iis_output.replace(gurobi_name, pyomo_name)

# Write the modified content back to the file
with open("iismodel-translated.ilp", "w") as file:
    file.write(iis_output)

print("File updated successfully!")
#print(iis_output)

File updated successfully!


In [312]:
#########################
# Decision Variables
#########################

# Indicates if preprocessing option k_prep is chosen for feedstock f at period t
model.Y_prep = pe.Var(model.F, model.K_prep, model.T, domain=pe.Binary)
# Indicates if pretreatment option k_pret is chosen for feedstock f at period t
model.Y_pret = pe.Var(model.F, model.K_pret, model.T, domain=pe.Binary)
# Indicates if fermentation option k_conv with strain j is chosen to produce final product p with component c at period t
model.Y_conv = pe.Var(model.C, model.P, model.K_conv, model.J, model.T, domain=pe.Binary)
# Indicates if purification option k_pur is chosen for product p at period t
model.Y_pur = pe.Var(model.P, model.K_pur, model.T, domain=pe.Binary)


In [314]:
# Total inbound flow of feedstock f at period t
model.F_in_feedstock = pe.Var(model.F, model.T, domain=pe.NonNegativeReals)
# Total outbound flow of final product p at period t
model.F_out_product = pe.Var(model.P, model.T, domain=pe.NonNegativeReals)


# Aggregated Variables
model.aggregation = pe.ConstraintList()

for t in model.T:
    for f in model.F:
        model.aggregation.add(model.F_in_feedstock[f, t] == 
                               sum(model.F_in_prep[f, k, t] for k in model.K_prep) 
                             #+ sum(model.F_in_pret(f, k, t) for f in model.F for k in model.K_pret)
                             #+ sum(model.F_in_conv(c, p, k, j, t) for c in model.C for p in model.P for k in model.K_conv for j in model.J)
                             #+ sum(model.F_in_pur(p, k, t) for p in model.P for k in model.K_pur)
                             )
    for p in model.P:
        model.aggregation.add(model.F_out_product[p, t] == 
                             #  sum(model.F_out_prep(f, k, t) for f in model.F for k in model.K_prep) 
                             #+ sum(model.F_out_pret(c, k, t) for c in model.C for k in model.K_pret)
                             #+ sum(model.F_out_conv(c, p, k, j, t) for c in model.C for p in model.P for k in model.K_conv for j in model.J)
                             sum(model.F_out_pur[p, k, t] for k in model.K_pur)
                             )



